In [37]:
import numpy as np
import random as rand


In [16]:
def check_optimality(gk,Hk,tolT): 
#   Revisa que se cumplan las condiciones de optimalidad: Grad=0, Hess positiva semidefinida

#   gk - Vector Gradiente de f en xk
#   Hk - Matriz Hessiana de f en xk
#   tolT - Tolerancia de el gradiente
    print(gk)
    return es_pos_def(Hk) and all(abs(gk)<tolT)


In [17]:
def Grad(f, xk, h):
#   Calcula el gradiende de f en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n = xk.size
    g = np.zeros(n)
    for i in range(0, n):
        b = np.copy(xk)
        b[i] += h
        g[i] = (f(b) - f(xk))/(h)
    return g

In [18]:
def Hess(f,xk,h):
#   Calcula el Hessiano en xk, con diferencia h

#   f - Función
#   xk - Punto a evaluar
#   h - Diferencia de aproximación

    n=xk.size
    H=np.zeros((n,n))
    for i in range(0,n):
        for j in range(0,n):
            ff=np.copy(xk)
            ff[i]+=h
            ff[j]+=h
            
            fb=np.copy(xk)
            fb[i]+=h
            fb[j]-=h
            
            bf=np.copy(xk)
            bf[i]-=h
            bf[j]+=h
            
            bb=np.copy(xk)
            bb[i]-=h
            bb[j]-=h
            
            H[i,j]=(f(ff)-f(fb)-f(bf)+f(bb))/(4*h**2)
    return H
            

In [19]:
def es_pos_def(A):
#   Prueba si la matriz A es positiva semidefinida si A es simétrica

#   A - Matriz a probar

        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        return False

In [20]:
def PruebaMod_Hess(Hk, Beta ) :
#   Modifica la Hessiana para que sea positiva semidefinida, según el algoritmo 3.3 de Nocedal Ed. 2

#   Hk - Matrizz hessiana a modificar
#   Beta - Valor arbitrario a aumentar

    n=int(np.sqrt(np.size(Hk)))
    tk=0
    v=np.matrix.diagonal(Hk)
    
    if min(v)<0:
        tk=-min(v)+Beta    
    

    while es_pos_def(Hk+np.eye(n)*tk)==False:
           tk= max(2*tk, Beta);
    
    return Hk+np.eye(n)*tk
     

In [21]:
def BactrackSearch(f,xk,pk,gk,alpha0,c,rho):
#   Busca el paso según el algoritmo de 3.1 de Nocedal Ed. 2

#   f - Función a evaluar
#   xk - punto alrededor del que se evalúa
#   pk -Dirección elegida previamente
#   alpha0 - Valor de paso máximo
#   c - peso sobre la derivada direccional
#   rho - factor de disminución del paso

    alpha_k=alpha0
   

    while f(xk+alpha_k*pk)> f(xk)+c*alpha_k*np.dot(gk,pk):
        
        alpha_k=alpha_k*rho
    return alpha_k
    

In [22]:
def Min_LineSearchNewtonHessMod(f , x0 , tolT , h1 ,h2 , alpha0, c, rho,maxit):  
#   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
#   con una aproximación de la Hessiana adaptada para garantizar que sea positiva definida.

#   f-Función
#   x0 - Punto inicial
#   tolT -  Tolerancia sobre condición de optimalidad
#   h1 -Diferencias para gradiente
#   h2 -Diferencias para Hessiana
#   c - peso sobre el gradiente para búsqueda lineal Bactracking
#   rho - factor de disminución del paso para búsqueda lineal Bactracking
#   maxit - Máximo número de Iteraciones

    xk=x0
    n=np.size(x0)
    for k in range(0,maxit):
    
    
        gk=Grad(f,xk,h1)                                       
        Hk=Hess(f,xk,h2)         
        
        if check_optimality(gk,Hk,tolT) :
            break
        
        HessMod=PruebaMod_Hess( Hk, 1e-3 ) 

        pk= -np.linalg.solve(HessMod,gk)
        
        
        alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
        xk_1=xk
        xk=xk+alpha_k*pk
        
        
        print(xk)
        print(f(xk))
            
    return [xk,k]

In [23]:
def Min_LineSearchNewton(f , x0 , tolT , h1 ,h2 , alpha0, c, rho,maxit):  
#   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de búsqueda lineal de Newton
#   con una aproximación de la Hessiana adaptada

#   f-Función
#   x0 - Punto inicial
#   tolT -  Tolerancia sobre condición de optimalidad
#   h1 -Diferencias para gradiente
#   h2 -Diferencias para Hessiana
#   c - peso sobre el gradiente para búsqueda lineal Bactracking
#   rho - factor de disminución del paso para búsqueda lineal Bactracking
#   maxit - Máximo número de Iteraciones

    xk=x0
    n=np.size(x0)
    for k in range(0,maxit):
    
    
        gk=Grad(f,xk,h1)                                       
        Hk=Hess(f,xk,h2)         
        
        if check_optimality(gk,Hk,tolT) :
            break
        
        pk= -np.linalg.solve(Hk,gk)
        
        
        alpha_k=BactrackSearch(f,xk, pk, gk, alpha0 ,c,  rho) 
        
        xk_1=xk
        xk=xk+alpha_k*pk
        
        
        print(xk)
        print(f(xk))
            
    return [xk,k]

In [24]:
def NewtonAlgorithm(f , x0 , tolT , h1 ,h2 ,maxit):  
#   Busca el mínimo de una función dada una aproximación inicial x0. Utiliza el método de Newton sin ninguna modificación o 
#   procedimiento extra.

#   f-Función
#   x0 - Punto inicial
#   tolT -  Tolerancia sobre condición de optimalidad
#   h1 -Diferencias para gradiente
#   h2 -Diferencias para Hessiana
#   c - peso sobre el gradiente para búsqueda lineal Bactracking
#   rho - factor de disminución del paso para búsqueda lineal Bactracking
#   maxit - Máximo número de Iteraciones

    xk=x0
    n=np.size(x0)
    for k in range(0,maxit):
    
    
        gk=Grad(f,xk,h1)                                       
        Hk=Hess(f,xk,h2)         
        
        if check_optimality(gk,Hk,tolT) :
            break
        
        pk= -np.linalg.solve(Hk,gk)
        
        
        xk_1=xk
        xk=xk+pk
        
        
        print(xk)
        print(f(xk))
            
    return [xk,k]

In [25]:

def fi(f,xk,pk,a):
    
    return(f(xk+a*pk))


def Inter(l,b):
    m = (l+b)/2
    
    return m

def Zoom(alo, ahi, f, xk, pk, c1, c2,  h, maxiter):
    
    a =np.zeros(1,maxiter+1)
    fi0 = fi(f,xk,pk,0)
    fid0 = Grad(fi(f,xk,pk,), 0, h)
    
    for i in range(0,maxiter):
        
        a[i] = Inter(alo, ahi)
        fid = Grad(fi(f,xk,pk,), a[i], h)
        fie = fi(f,xk,pk,a[i])    
    
        if (fie>fi0 + c1*a[i]*fid0):
            ahi = a[i]    
            
        else:
            if abs(fid)<= -c2*fid0:
                ac = a[i] 
                break
            if fid*(ahi-alo)>= 0:
                
                ahi = alo
            alo = a[i]    
                
            return ac  

def LineSearchWolf(f,a_max, xk, pk, c1, c2, gk, maxiter):
    
    
    a =np.zeros(1,maxiter+1)
    fi0 = fi(f,xk,pk,0)
    fid0 = Grad(fi(f,xk,pk,), 0, h)
    
    for i in range(0,maxiter):
        
        a[1]= np.random.rand(a[i-1],a_max)
        
        
        fie = fi(f,xk,pk,a[i])
        fid = Grad(fi(f,xk,pk,), a[i], h)
        
        
        if fie > (fi0 + c1*a[i]*fid0):
            a[i+1] = Zoom(a[i-1],a[i], f, xk, pk,c1, c2, h, maxiter)
            break
        
        if abs(fid) <= -c2*fid0:
            a[i+1] = a[i] 
            break
                    
        if fid >= 0:
            a[i+1] = Zoom(a[i],a[i-1], f, xk, pk, c1, c2, h, maxiter)
            break
        
            return a[i+1]

In [ ]:
def LineSearchWolf(f,a_max, xk, c1, c2, gk, maxiter)
    alpha0=0
    alpha1=0.75(a_max)

In [26]:
def BFGS(f,x0,epsilon,maxit,h1,h2):
#Busca el mínimo de la función con un método Quasinewton y una matriz aproximada con BFGS 
#   f- Función
#   x0- Punto inicial
#   epsilon- Tolerancia para la condición del gradiente
#   h1- diferencia para la primera aproximación de Hessiana
#   h2- diferencia para la aproximación del Gradiente

    n=np.size(x0)
    Hk=np.linalg.inv(Hess(f,x0,h1))
    gk=Grad(f,x0,h2)
    print(gk)
    xk=x0
    for k in range(0 ,maxit):
        if all(abs(gk)<epsilon):
            break
        pk=np.matmul(-Hk,gk)
            
        #alphak=BactrackSearch(f,xk,pk,gk,1,0.8,0.9)
        alphak= LineSearchWolf(f,1, xk, pk, 0.9, 0.8, 0.00001, 20)
            
        xk_1=alphak*pk+xk
        gk_1=Grad(f,xk_1,h2)
        sk= xk_1  - xk
        yk= gk_1-gk
        
        rhok=1/np.dot(yk,sk)
        print(rhok)    
        Hk= np.matmul(np.matmul((np.identity(n)-rhok*np.matmul(sk,np.transpose(yk))),Hk),(np.identity(n)-rhok*np.matmul(yk,np.transpose(sk))))
        gk=gk_1
        xk=xk_1
        print(xk)
        print(f(xk))

    return [f(xk),xk,k]

In [27]:
def rosenbrock1_100(v):
    # a=1 b=100
    return ((1-v[0])**2+100*(v[1]-v[0]**2)**2)


In [28]:
def rosenbrock100_1(v):
    # a=100 b=1
    return ((100-v[0])**2+1*(v[1]-v[0]**2)**2)

In [29]:
def rosenbrock50_50(v):
    # a=50 b=50
    return ((50-v[0])**2+50*(v[1]-v[0]**2)**2)


In [30]:
def rosenbrock5_60(v):
    # a=1 b=100
    return ((5-v[0])**2+60*(v[1]-v[0]**2)**2)

In [31]:
def rosenbrock1_1(v):
    # a=1 b=100
    return ((1-v[0])**2+1*(v[1]-v[0]**2)**2)

In [32]:
import csv
import numpy

#En este script obtenemos los datos de nuestra función

with open('crime.csv', 'r') as file:
    reader = csv.reader(file)
    n=31057

    rows=[]
    data=[]
    for row in reader: 
        rows.append(row)

for i in range(1,3057):
    data.append(numpy.asarray(rows[i][3:],dtype=float))
    


In [33]:
def costo_camaras(c,data, delta, sigma, N, M):
#Calcula el Costo de posicionar un número N de cámaras para cubrir M crimen. Hay uncosto cuadrático para la distancia 
#entre una cámara y un crimen. Y también hay un costo inverso a la distancia al cuadrado entre cámaras. Lo que implica
#una serie de singularidades para todos los puntos donde dos cámaras son iguales, por lo que esta no puede ser condición 
#inicial.

    costo=0

#   c-Lista de arrays, que indican la localización de las cámaras,   N arrays
#   data-Lista de arrays, que indican la localicación de los crímenes, M arrays
#   delta= peso de distancia camaras crimenes
#   sigma= peso del inverso de la distancia camara camara
    
    #Primero el costo de distancia de cámaras a crímenes
    for i in range(1,N):
        for j in range(1,M):
            costo=delta*numpy.linalg.norm(c[i]-data[j])^2
   
    #Luego el costo de cercanía de las cámaras
    for i in range(1,N):
        for j in range(1,N):
            if i!=j:
                costo=costo+sigma/numpy.linalg.norm(c[i]-c[j])^(-2)
    
    #Regresamos finalmente el costo total:
    
    return costo

In [34]:
c=

[array([ 19.31871412, -99.25441841]),
 array([ 19.37876233, -99.20548829]),
 array([ 19.3479712, -99.0301617]),
 array([ 19.55731088, -99.13416645]),
 array([ 19.34212519, -99.12427974]),
 array([ 19.33472166, -99.03031684]),
 array([ 19.4473462, -99.1591203]),
 array([ 19.40373446, -99.13849869]),
 array([ 19.56340479, -99.13257004]),
 array([ 19.39297696, -99.10110124]),
 array([ 19.3335839, -99.1652134]),
 array([ 19.32375056, -99.10529656]),
 array([ 19.41086184, -99.06806615]),
 array([ 19.3604001, -99.1465168]),
 array([ 19.39621487, -99.19032741]),
 array([ 19.34771317, -99.26916303]),
 array([ 19.4906426, -99.1741282]),
 array([ 19.32921274, -99.15460307]),
 array([ 19.51481528, -99.13314474]),
 array([ 19.287941 , -99.0680133]),
 array([ 19.44310411, -99.16417994]),
 array([ 19.3586505, -99.1408242]),
 array([ 19.4565703, -99.1158829]),
 array([ 19.4344073, -99.2192733]),
 array([ 19.4276057, -99.1571211]),
 array([ 19.32685897, -99.05689451]),
 array([ 19.38155075, -99.213333

In [35]:

BFGS(rosenbrock1_1,np.array([0.999999448753295, 3.450318174756265]),0.00000000000000000001,100,0.0001,0.000001)

[-9.80127271  4.90063955]


TypeError: Cannot interpret '21' as a data type

In [36]:
NewtonAlgorithm(rosenbrock1_100 , np.array([0.999999448753295, 3.450318174756265]) , 0.000001 , 0.00000001 ,0.00001 ,100)

[-980.12717444  490.0638487 ]
[0.99999998 0.99999846]
2.2637492853284715e-10
[ 0.00060581 -0.00029992]
[0.999997   0.99999399]
9.027140868248177e-12
[ 5.33207512e-09 -1.78356220e-09]


[array([0.999997  , 0.99999399]), 2]

In [110]:
Min_LineSearchNewton(rosenbrock1_100 , np.array([0.999999448753295, 3.450318174756265]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[-980.1271744436235  490.0638487015385]
[0.99999965588873  2.501014113525679]
225.30454350668904
[-600.4057155450937   300.20296151178627]
[0.999999652115163 1.919490056754497]
84.5463243977244
[-367.7961743164815   183.89814897545875]
[0.999999679264299 1.563260523236901]
31.726313966843843
[-225.30439274248693  112.65223278655867]
[0.999999718826148 1.345041666087074]
11.905413940318413
[-138.01685057046598   69.00844606860801]
[0.999999732474359 1.21136525999771 ]
4.467549931666303
[-84.5462924559115   42.27315981708557]
[0.999999754301319 1.129477862823531]
1.6764644211736017
[-51.791325827998946  25.895671673659137]
[0.999999792188996 1.079315369519273]
0.6290993772571206
[-31.726304106083347  15.863157931672589]
[0.99999985992006  1.048586942950334]
0.23607182495599155
[-19.434882914115548   9.717445562107407]
[0.999999981749334 1.029763499054487]
0.08858680487825388
[-11.905410177148745   5.952708075074309]
[1.00000020676248  1.018232943716105]
0.03324251571699116
[-7.2930091472

[array([0.999996994994272, 0.999994009839379]), 99]

In [111]:
Min_LineSearchNewtonHessMod(rosenbrock1_100 , np.array([0.999999448753295, 3.450318174756265]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[-980.1271744436235  490.0638487015385]
[1.825769266489945 3.604228840799707]
8.01491117417121
[-196.11243278205848   54.15908592709684]
[2.174726716742146 4.727537985098286]
1.380343216131507
[ 4.00079229745387  -0.379660458627029]
[2.04692628637584  4.171960890675234]
1.128261729391077
[16.787796019990253 -3.589265218373328]
[1.948724868189653 3.777664066816967]
0.9395388906601674
[17.381678141248358 -3.972907991478536]
[1.85747517607178  3.431524126491043]
0.7701949258609174
[15.6013764907037   -3.737979614726328]
[1.770912580576065 3.118887702506064]
0.6240406070039725
[13.756647332385796 -3.448732088795481]
[1.688028506478551 2.833574392611486]
0.49855573288743593
[12.088868517334816 -3.173168194869902]
[1.609171056852125 2.574935729540482]
0.39210208424682136
[10.548815787547738 -2.899151119173382]
[1.534445096858504 2.341375300352392]
0.30291448925432746
[ 9.137904966038946 -2.629289969080517]
[1.464010849441981 2.131508235226965]
0.22927620220419406
[ 7.849599523157558 -2.36390

[1.000009878111354 1.000019238536041]
1.2438713619299023e-10
[ 0.000230882044208 -0.000102556848079]
[1.000009878091457 1.000019238497039]
1.243866610697467e-10
[ 0.000230881689898 -0.000102556689628]
[1.00000987806935  1.000019238453705]
1.2438613316730194e-10
[ 0.000230881293606 -0.000102556513591]
[1.000009878047243 1.00001923841037 ]
1.243856052659896e-10
[ 0.000230880897314 -0.000102556337554]
[1.000009878036669 1.000019238389643]
1.243853527729735e-10
[ 0.000230880707766 -0.000102556253355]
[1.000009878033351 1.000019238383139]
1.2438527353620665e-10
[ 0.000230880645894 -0.000102556226931]
[1.000009878022777 1.000019238362412]
1.243850210435309e-10
[ 0.000230880456347 -0.000102556142732]
[1.000009878012203 1.000019238341685]
1.2438476855111423e-10
[ 0.000230880266799 -0.000102556058533]
[1.000009878001629 1.000019238320959]
1.2438451605895658e-10
[ 0.000230880079639 -0.000102555974334]
[1.000009877983723 1.000019238285858]
1.2438408845890227e-10
[ 0.00023087975624  -0.00010255583

[array([1.000009877911799, 1.000019238144873]), 99]

In [86]:
Min_LineSearchNewton(rosenbrock50_50 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.6, 0.9,10000)

[4. 5.]
8166.0
[ 4.03049239 13.26302742]
2557.764561047093
[ 4.15494694 16.68154299]
2118.707479299791
[ 4.92926949 23.69811686]
2049.345608168345
[ 5.66942087 31.59459858]
1980.200896021119
[ 6.46483447 41.16145277]
1915.321798179827
[ 7.14225906 50.55295523]
1847.315842785891
[ 7.96484046 62.71614744]
1793.057558439701
[ 8.5385842  72.57823028]
1724.4672995227609
[ 9.26027409 85.09702634]
1681.2191036253212
[ 9.87309826 97.10257319]
1617.218109596999
[ 10.63391441 112.39961114]
1572.8443717702635
[ 11.20384594 125.20133128]
1510.4173822518017
[ 11.96693421 142.51358592]
1470.5909297111182
[ 12.5071744  156.13754577]
1409.9712490050465
[ 13.23573578 174.53425996]
1372.764849052082
[ 13.79397006 189.96206218]
1315.729709066102
[ 14.53719221 210.67053951]
1279.3523343875613
[ 15.06656415 226.721106  ]
1224.2719220811643
[ 15.77006048 248.08499739]
1190.2821830998266
[ 16.32200858 266.10330415]
1138.8479894426905
[ 17.01644897 288.97236642]
1105.1530088004413
[ 17.57137706 308.44544004]


[array([  49.99747525, 2499.74753129]), 107]

In [87]:
Min_LineSearchNewton(rosenbrock5_60 , np.array([4,5]) , 0.000001 , 0.00000001 ,0.00001 , 1, 0.8, 0.9,100)

[4. 5.]
7261.0
[4.0003118  9.26416532]
2725.3041810878503
[ 4.00079244 11.87858054]
1023.3023772363298
[ 4.00156319 13.48391686]
384.62325842227614
[ 4.00283366 14.47371255]
144.9518469596896
[ 4.00489986 15.09035418]
55.0113334837224
[ 4.0082564  15.48485062]
21.255956159778975
[ 4.01368725 15.75358254]
8.581363318763078
[ 4.02242485 15.96168405]
3.8127882133546604
[ 4.03635579 16.15829838]
2.0038758474803964
[ 4.05823359 16.38677544]
1.2951452006915107
[ 4.09171187 16.69045694]
0.9850450285541313
[ 4.1460305  17.15720254]
0.7921187960531635
[ 4.22965975 17.86614215]
0.6276378833439636
[ 4.32497398 18.68385714]
0.4835056754650716
[ 4.41502648 19.4731104 ]
0.3646556609654345
[ 4.49925129 20.22607424]
0.26847479209461883
[ 4.57745193 20.93798384]
0.19219549863042779
[ 4.6421812  21.53706656]
0.13783364232354717
[ 4.7029719  22.10697069]
0.09545142403695245
[ 4.75815319 22.63072677]
0.06367373806565499
[ 4.80736101 23.10300464]
0.040681274458405456
[ 4.85041363 23.52026476]
0.02471804093

[array([ 4.999967  , 24.99966995]), 41]